In [35]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re
import warnings
warnings.filterwarnings('ignore')
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import json
from datetime import datetime, timedelta
import os
chromedriver = "/Users/stevenwang/Projects/funded-and-hiring-site/newsletter_scraper/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
import pyperclip
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pytest_timeout
import dateparser

In [72]:
def get_content(url):
    page = requests.get(url)
    return page.content

In [73]:
ttnews_content = get_content('https://www.ttnews.com/articles')
fw_news_content = get_content('https://www.freightwaves.com/news/category/trucking')

In [79]:
def get_ttnews(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='views-row')
    
    for article in articles:
        title = article.find('a')
        date = article.find(class_='date-display-single')
        text =  article.find('p') # description
        image = article.find('img')

        if (date != None and title != None and text != None):
            parsed_date = dateparser.parse(date['content'])
            companies.append('TTNews')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append(title.get('href'))
            titles.append(title.getText().strip())
            images.append(image)
            company_urls.append('https://ttnews.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

ttnews = get_ttnews(ttnews_content)

ttnews

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,"No Vote Held on Infrastructure, but Talks to R...",WASHINGTON — Despite a long night of frantic n...,2021-10-01,/articles/no-vote-held-infrastructure-talks-re...,https://ttnews.com/,[]
1,TTNews,TT Newsmakers,"Sgt. Zach Heard, Illinois State Police, and Ke...",2021-09-30,/newsmakers,https://ttnews.com/,[]
2,TTNews,ATRI Report Urges Electric Tax to Help Fund Hi...,A new report from the American Transportation ...,2021-09-30,/articles/atri-report-urges-electric-tax-help-...,https://ttnews.com/,[]
3,TTNews,Transportation Stakeholders Press Congress on ...,To guarantee long-term funding for the country...,2021-09-30,/articles/transportation-stakeholders-press-co...,https://ttnews.com/,[]
4,TTNews,Congress Clears Government Funding Bill to Ave...,The House passed a nine-week spending bill to ...,2021-09-30,/articles/senate-passes-funding-bill-avert-gov...,https://ttnews.com/,[]
5,TTNews,Transport Topics Radio,Transport Topics Radio is a weekly news roundt...,2021-09-30,/transport-topics-radio,https://ttnews.com/,[]
6,TTNews,Port Leaders Warn of Supply Chain Challenges a...,"AUSTIN, Texas — The leader of one of the natio...",2021-09-30,/articles/port-leaders-warn-supply-chain-chall...,https://ttnews.com/,[]
7,TTNews,Transportation Stakeholders Focus on Potential...,Federal lawmakers are working on numerous regu...,2021-09-30,/articles/transportation-stakeholders-focus-po...,https://ttnews.com/,[]
8,TTNews,FMCSA Advised to Give Drivers 90 Days to Repla...,A medical advisory board to the Federal Motor ...,2021-09-30,/articles/fmcsa-advised-give-drivers-90-days-r...,https://ttnews.com/,[]
9,TTNews,Lordstown to Sell Ohio Plant to Foxconn in $28...,Lordstown Motors Corp. agreed to partner with ...,2021-09-30,/articles/lordstown-nears-deal-sell-ohio-plant...,https://ttnews.com/,[]


In [75]:
def get_freightWaves(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []

    articles = soup.find_all(class_='post-item')
    
    for article in articles:
        title = article.find('a')
        date =  article.find(class_='date meta-item tie-icon')
        text =  article.find('p')
        image = article.find('img')

        companies.append('FreightWaves')
        description.append(text.getText())
        publish_date.append(dateparser.parse(date.get_text()).date())
        stories.append(title.get('href'))
        titles.append(title.get('aria-label'))
        images.append(image)
        company_urls.append('https://www.freightwaves.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe
        
fw_news = get_freightWaves(fw_news_content)

fw_news

,company,headline,description,publish_date,story_url,company_url,image
0,FreightWaves,Lordstown Motors may sell its plant to Taiwan’...,Taiwan’s Foxconn would manufacture Lordstown M...,2021-09-30,https://www.freightwaves.com/news/lordstown-mo...,https://www.freightwaves.com/,[]
1,FreightWaves,Ransomware attack on Forward Air may have expo...,Nine months after a ransomware attack on Forwa...,2021-09-30,https://www.freightwaves.com/news/ransomware-a...,https://www.freightwaves.com/,[]
2,FreightWaves,Workhorse executive housecleaning continues wi...,"New CEO continues remaking executive team, wit...",2021-09-30,https://www.freightwaves.com/news/workhorse-ex...,https://www.freightwaves.com/,[]
3,FreightWaves,FreightWaves Classics: I-24 travels through fo...,I-24 passes through four states and provides a...,2021-09-30,https://www.freightwaves.com/news/freightwaves...,https://www.freightwaves.com/,[]
4,FreightWaves,M&A activity staying hot across logistics indu...,Pent-up demand is making its way to the surfac...,2021-09-30,https://www.freightwaves.com/news/ma-activity-...,https://www.freightwaves.com/,[]
5,FreightWaves,ArcBest sees billions of dollars in opportunit...,ArcBest’s acquisition of truckload broker MoLo...,2021-09-30,https://www.freightwaves.com/news/arcbest-sees...,https://www.freightwaves.com/,[]
6,FreightWaves,"Be prepared, attorneys say: More driver classi...",A pair of lawyers tell a TCA audience that tak...,2021-09-30,https://www.freightwaves.com/news/be-prepared-...,https://www.freightwaves.com/,[]
7,FreightWaves,Successful driver recruitment is multifaceted ...,Tenstreet partnered with FreightWaves to bette...,2021-09-30,https://www.freightwaves.com/news/successful-d...,https://www.freightwaves.com/,[]
8,FreightWaves,"Pick your bottleneck: Ports, chassis, containe...",With disruptions likely to linger well into 20...,2021-09-30,https://www.freightwaves.com/news/pick-your-bo...,https://www.freightwaves.com/,[]
9,FreightWaves,Report: Electric vehicle tax could revive High...,An ATRI study shows how electric vehicles can ...,2021-09-29,https://www.freightwaves.com/news/report-elect...,https://www.freightwaves.com/,[]


In [80]:
final = pd.concat([ttnews, fw_news], axis = 0)
#replace all .coms in company names
final['company'] = final.company.str.replace("[.].*|•|’|,|[+]|[()]|[#]", '').str.strip()

In [81]:
final

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,"No Vote Held on Infrastructure, but Talks to R...",WASHINGTON — Despite a long night of frantic n...,2021-10-01,/articles/no-vote-held-infrastructure-talks-re...,https://ttnews.com/,[]
1,TTNews,TT Newsmakers,"Sgt. Zach Heard, Illinois State Police, and Ke...",2021-09-30,/newsmakers,https://ttnews.com/,[]
2,TTNews,ATRI Report Urges Electric Tax to Help Fund Hi...,A new report from the American Transportation ...,2021-09-30,/articles/atri-report-urges-electric-tax-help-...,https://ttnews.com/,[]
3,TTNews,Transportation Stakeholders Press Congress on ...,To guarantee long-term funding for the country...,2021-09-30,/articles/transportation-stakeholders-press-co...,https://ttnews.com/,[]
4,TTNews,Congress Clears Government Funding Bill to Ave...,The House passed a nine-week spending bill to ...,2021-09-30,/articles/senate-passes-funding-bill-avert-gov...,https://ttnews.com/,[]
5,TTNews,Transport Topics Radio,Transport Topics Radio is a weekly news roundt...,2021-09-30,/transport-topics-radio,https://ttnews.com/,[]
6,TTNews,Port Leaders Warn of Supply Chain Challenges a...,"AUSTIN, Texas — The leader of one of the natio...",2021-09-30,/articles/port-leaders-warn-supply-chain-chall...,https://ttnews.com/,[]
7,TTNews,Transportation Stakeholders Focus on Potential...,Federal lawmakers are working on numerous regu...,2021-09-30,/articles/transportation-stakeholders-focus-po...,https://ttnews.com/,[]
8,TTNews,FMCSA Advised to Give Drivers 90 Days to Repla...,A medical advisory board to the Federal Motor ...,2021-09-30,/articles/fmcsa-advised-give-drivers-90-days-r...,https://ttnews.com/,[]
9,TTNews,Lordstown to Sell Ohio Plant to Foxconn in $28...,Lordstown Motors Corp. agreed to partner with ...,2021-09-30,/articles/lordstown-nears-deal-sell-ohio-plant...,https://ttnews.com/,[]


In [60]:
final.head()

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,"No Vote Held on Infrastructure, but Talks to R...",WASHINGTON — Despite a long night of frantic ...,2021-10-01,/articles/no-vote-held-infrastructure-talks-re...,https://ttnews.com/,[]
1,TTNews,TT Newsmakers,"Sgt. Zach Heard, Illinois State Police, and K...",2021-09-30,/newsmakers,https://ttnews.com/,[]
2,TTNews,ATRI Report Urges Electric Tax to Help Fund Hi...,A new report from the American Transportation...,2021-09-30,/articles/atri-report-urges-electric-tax-help-...,https://ttnews.com/,[]
3,TTNews,Transportation Stakeholders Press Congress on ...,To guarantee long-term funding for the countr...,2021-09-30,/articles/transportation-stakeholders-press-co...,https://ttnews.com/,[]
4,TTNews,Congress Clears Government Funding Bill to Ave...,The House passed a nine-week spending bill to...,2021-09-30,/articles/senate-passes-funding-bill-avert-gov...,https://ttnews.com/,[]


In [61]:
len(final)

30

In [44]:
#check previous posts, check to see there are no duplicates
# old = pd.read_csv('historical_deals.csv')
# dupes = pd.merge(final, old[['company', 'deal_size']], how = 'inner', on = ['company', 'deal_size'])
# final = final[~final.company.isin(dupes.company)]

In [62]:
len(final)

30

In [63]:
final

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,"No Vote Held on Infrastructure, but Talks to R...",WASHINGTON — Despite a long night of frantic ...,2021-10-01,/articles/no-vote-held-infrastructure-talks-re...,https://ttnews.com/,[]
1,TTNews,TT Newsmakers,"Sgt. Zach Heard, Illinois State Police, and K...",2021-09-30,/newsmakers,https://ttnews.com/,[]
2,TTNews,ATRI Report Urges Electric Tax to Help Fund Hi...,A new report from the American Transportation...,2021-09-30,/articles/atri-report-urges-electric-tax-help-...,https://ttnews.com/,[]
3,TTNews,Transportation Stakeholders Press Congress on ...,To guarantee long-term funding for the countr...,2021-09-30,/articles/transportation-stakeholders-press-co...,https://ttnews.com/,[]
4,TTNews,Congress Clears Government Funding Bill to Ave...,The House passed a nine-week spending bill to...,2021-09-30,/articles/senate-passes-funding-bill-avert-gov...,https://ttnews.com/,[]
5,TTNews,Transport Topics Radio,Transport Topics Radio is a weekly news round...,2021-09-30,/transport-topics-radio,https://ttnews.com/,[]
6,TTNews,Port Leaders Warn of Supply Chain Challenges a...,"AUSTIN, Texas — The leader of one of the nati...",2021-09-30,/articles/port-leaders-warn-supply-chain-chall...,https://ttnews.com/,[]
7,TTNews,Transportation Stakeholders Focus on Potential...,Federal lawmakers are working on numerous reg...,2021-09-30,/articles/transportation-stakeholders-focus-po...,https://ttnews.com/,[]
8,TTNews,FMCSA Advised to Give Drivers 90 Days to Repla...,A medical advisory board to the Federal Motor...,2021-09-30,/articles/fmcsa-advised-give-drivers-90-days-r...,https://ttnews.com/,[]
9,TTNews,Lordstown to Sell Ohio Plant to Foxconn in $28...,Lordstown Motors Corp. agreed to partner with...,2021-09-30,/articles/lordstown-nears-deal-sell-ohio-plant...,https://ttnews.com/,[]


In [82]:
#save deal records
# new_old = pd.concat([final, old], axis = 0)
new_old = pd.concat([final], axis = 0)

new_old.to_csv('historical_deals.csv', index = False)

In [65]:
len(new_old)

30

In [66]:
date = 'Sunday, 08/29/2021'

title = '''
<h1>
  Funded & Hiring
</h1>
<h2 style="text-align:center">
  <strong>{date}</strong>
</h2>
<h2>
  A weekly roundup of funded startups and the jobs they're hiring for.
</h2>
<h4>
  Forwarded from a friend? 
  <a href = 'https://fundedandhiring.com/'>Subscribe</a> to stay up to date on the latest startup funding and job alerts.
</h4>
'''.format(date = date).strip()

In [67]:
print(title)
pyperclip.copy(title)

<h1>
  Funded & Hiring
</h1>
<h2 style="text-align:center">
  <strong>Sunday, 08/29/2021</strong>
</h2>
<h2>
  A weekly roundup of funded startups and the jobs they're hiring for.
</h2>
<h4>
  Forwarded from a friend? 
  <a href = 'https://fundedandhiring.com/'>Subscribe</a> to stay up to date on the latest startup funding and job alerts.
</h4>


PythonQtError: No Qt bindings could be found

In [68]:
content = ''''''

final_dict = final.to_dict(orient = 'records')
final_dict = final_dict[0:7]
for row in final_dict:
    headline = row['headline']
    company_name = row['company']
    publish_date = row['publish_date']
    description = row['description']
    story_url = row['story_url']
    company_url = row['company_url']
    img = row['image']

    row_url = company_url + story_url
    
    entry = '''
    <tr>
      <td>
        {img}
        <h3>
          <a href = "{row_url}">{headline}</a>
        </h3>
        <p>
          {description}
        </p>
        <p>
          <a href = "{company_url}">{company_name}</a>
        </p>
        <p>
          <b>{publish_date}</b>
        </p>
      </td>
    </tr> '''.format(row_url = row_url, headline = headline, company_url = company_url, company_name = company_name, description = description, publish_date = publish_date, img=img).strip()

    content += '\n' + entry.strip()

In [70]:
pyperclip.copy(content)
print(content)

PythonQtError: No Qt bindings could be found

In [ ]:
#job boards
# stackoverflow, jobvite, bamboohr